In [1]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List, Dict


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("[INFO] Using device:", DEVICE)


In [ ]:
def read_image(path: Path) -> np.ndarray:
    img = cv2.imread(str(path))
    if img is None:
        raise ValueError(f"Failed to read image: {path}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


In [ ]:
# Requires: pip install ultralytics
from ultralytics import YOLO

MODEL_PATH = "../outputs/models/damage/yolo_damage.pt"  # custom later

if Path(MODEL_PATH).exists():
    model = YOLO(MODEL_PATH)
    print("✅ Loaded custom damage detection model")
else:
    model = YOLO("yolov8n.pt")  # fallback pretrained
    print("⚠️ Using generic YOLOv8 model")


In [ ]:
DAMAGE_CLASSES = [
    "floor_damage",
    "door_damage",
    "structural_damage"
]


In [ ]:
@torch.no_grad()
def detect_damage(img: np.ndarray, conf_thresh=0.25) -> List[Dict]:
    """
    Returns list of detected damage regions.
    """
    results = model(img, conf=conf_thresh, verbose=False)

    detections = []
    for r in results:
        for box in r.boxes:
            detections.append({
                "bbox": box.xyxy[0].cpu().numpy(),
                "confidence": float(box.conf),
                "class_id": int(box.cls)
            })

    return detections


In [ ]:
def visualize_detections(img: np.ndarray, detections: List[Dict]):
    vis = img.copy()

    for d in detections:
        x1, y1, x2, y2 = map(int, d["bbox"])
        conf = d["confidence"]

        cv2.rectangle(vis, (x1,y1), (x2,y2), (255,0,0), 2)
        cv2.putText(
            vis,
            f"{conf:.2f}",
            (x1, y1-5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255,0,0),
            1
        )

    plt.figure(figsize=(6,4))
    plt.imshow(vis)
    plt.axis("off")
    plt.show()


In [ ]:
def compare_damage_detection(
    raw_img: np.ndarray,
    enhanced_img: np.ndarray
):
    raw_det = detect_damage(raw_img)
    enh_det = detect_damage(enhanced_img)

    print(f"Raw detections: {len(raw_det)}")
    print(f"Enhanced detections: {len(enh_det)}")

    print("\nRaw frame:")
    visualize_detections(raw_img, raw_det)

    print("\nEnhanced frame:")
    visualize_detections(enhanced_img, enh_det)


In [ ]:
def avg_confidence(detections: List[Dict]) -> float:
    if len(detections) == 0:
        return 0.0
    return np.mean([d["confidence"] for d in detections])


In [ ]:
def compare_confidence(raw_img, enhanced_img):
    raw_det = detect_damage(raw_img)
    enh_det = detect_damage(enhanced_img)

    print(
        f"Avg confidence → Raw: {avg_confidence(raw_det):.3f} | "
        f"Enhanced: {avg_confidence(enh_det):.3f}"
    )


In [ ]:
def batch_damage_analysis(
    image_dir: Path,
    enhanced_dir: Path,
    max_images=20
):
    deltas = []

    for img_path in list(image_dir.glob("*.jpg"))[:max_images]:
        raw = read_image(img_path)
        enhanced = read_image(enhanced_dir / img_path.name)

        raw_conf = avg_confidence(detect_damage(raw))
        enh_conf = avg_confidence(detect_damage(enhanced))

        deltas.append(enh_conf - raw_conf)

    print("Mean confidence improvement:", np.mean(deltas))
